In [ ]:
# Collision Checks
import matplotlib.pyplot as plt
import pyfvks
from fvks.visualization.draw_dispatch import draw_object

# --------------------- Collision check between basic shapes ------------------------#

# Oriented rectangle with width/2, height/2, orientation, x-position , y-position
obb = pyfvks.collision.RectOBB(1, 2, 0.3, 8, 10)

# Axis-aligned rectangle with width/2, height/2, x-position , y-position
aabb = pyfvks.collision.RectAABB(2, 3, 3, 2)

# Circle with radius, x-position , y-position
circ = pyfvks.collision.Circle(2.5, 6, 7)

print('Collision between OBB and AABB: ', obb.collide(aabb))
print('Collision between AABB and Circle: ', aabb.collide(circ))
print('Collision between Circle and OBB:  ', circ.collide(obb))

plt.figure(figsize=(10, 10))

draw_object([obb, aabb, circ])
# set plotting region
plt.axes().set_aspect('equal')
plt.ylim([-5, 20])
plt.xlim([-5, 20])
plt.show()


In [ ]:
# --------------------- Collision check using shapegroups ----------------------------#

sg = pyfvks.collision.ShapeGroup()
sg.add_shape(obb)
sg.add_shape(aabb)

print('Collision between Circle and Shapegroup: ', circ.collide(sg))

plt.figure(figsize=(10, 10))

draw_object([sg, circ])
plt.axes().set_aspect('equal')
plt.ylim([-5, 20])
plt.xlim([-5, 20])
plt.show()


In [ ]:

# ----------------- Collision checks between time varying obstacles ------------------#

tvo1 = pyfvks.collision.TimeVariantCollisionObject(1)
tvo1.append_obstacle(pyfvks.collision.RectOBB(2, 1, 0.0, 2.0, 5)) # time step 1
tvo1.append_obstacle(pyfvks.collision.RectOBB(2, 1, 0.0, 2.5, 5)) # time step 2
tvo1.append_obstacle(pyfvks.collision.RectOBB(2, 1, 0.0, 3, 5))   # time step 3
tvo1.append_obstacle(pyfvks.collision.RectOBB(2, 1, 0.0, 3.5, 5)) # time step 4
tvo1.append_obstacle(pyfvks.collision.RectOBB(2, 1, 0.0, 4, 5))   # time step 5
tvo1.append_obstacle(pyfvks.collision.RectOBB(2, 1, 0.0, 4.5, 5)) # time step 6
tvo1.append_obstacle(pyfvks.collision.RectOBB(2, 1, 0.0, 5, 5))   # time step 7
tvo1.append_obstacle(pyfvks.collision.RectOBB(2, 1, 0.0, 5.5, 5)) # time step 8

# Time variant obstacle that starts at time 4
tvo2 = pyfvks.collision.TimeVariantCollisionObject(4)
tvo2.append_obstacle(pyfvks.collision.RectOBB(2, 1, 1.5, 6.0, 0)) # time step 4
tvo2.append_obstacle(pyfvks.collision.RectOBB(2, 1, 1.5, 6.0, 2)) # time step 5
tvo2.append_obstacle(pyfvks.collision.RectOBB(2, 1, 1.5, 6.0, 3)) # time step 6
tvo2.append_obstacle(pyfvks.collision.RectOBB(2, 1, 1.5, 6.0, 4)) # time step 7
tvo2.append_obstacle(pyfvks.collision.RectOBB(2, 1, 1.5, 6.0, 5)) # time step 8
tvo2.append_obstacle(pyfvks.collision.RectOBB(2, 1, 1.5, 6.0, 6)) # time step 9
tvo2.append_obstacle(pyfvks.collision.RectOBB(2, 1, 1.5, 6.0, 7)) # time step 10

print('Collision between time-varying obstacle tvo1 and tvo2: ', tvo1.collide(tvo2))

plt.figure(figsize=(10, 10))

draw_object([tvo1, tvo2])
plt.axes().set_aspect('equal')
plt.ylim([-5, 20])
plt.xlim([-5, 20])
plt.show()


In [ ]:
cc = pyfvks.collision.CollisionChecker()
cc.add_collision_object(tvo1)
cc.add_collision_object(pyfvks.collision.RectOBB(2, 1, 1.5, 6.0, 0))
print('Collision with trajectory: ', cc.collide(tvo1))

In [ ]:
# ----------------------- Collision checker from scenario ----------------------------#
import os
import numpy as np

from fvks.scenario.commonroad.file_reader import CommonRoadFileReader

filename = '/commonroad/scenarios/NGSIM/Lankershim/NGSIM_Lanker_1.xml'
scenario = CommonRoadFileReader(os.getcwd() + filename).open_scenario()

cc = scenario.create_collision_checker()
rect = pyfvks.collision.RectOBB(5.0, 5.0, 1.5, 2.0, 10)

print('Collision between scenario and rectangle: ', cc.collide(rect))

plt.figure(figsize=(10, 10))
draw_object(scenario)
draw_object(cc)
draw_object(rect)
plt.show()

In [ ]:
from fvks.scenario.trajectory import Trajectory, collision_object_from_trajectory
from fvks.scenario.scenario import SimulatedCar

position = np.array([[0, 0], [5, 0], [10, 0], [15, 2], [18, 8]])
orientation = np.array([0, 0.2, 0.5, 0.8, 1.4])
# starting at time 17 with length 4.8 and width 1.8
trajectory = Trajectory.create_from_vertices(position, 0, orientation) 
trajectory_collision_object = collision_object_from_trajectory(trajectory,
                                                               CAR_LENGTH=4.0, CAR_WIDTH=1.8)

car = SimulatedCar(trajectory, scenario.dt, car_id=1000, length=4.0, width=1.8)
car_collision_object = car.create_collision_object()

rect = pyfvks.collision.RectOBB(5.0, 5.0, 1.5, 2.0, 10)
print('Collision between trajectory and rectangle: ', trajectory_collision_object.collide(rect))
print('Collision between car and rectangle: ', car_collision_object.collide(rect))

plt.figure(figsize=(10, 10))

plt.ylim([-5, 20])
plt.xlim([-5, 20])
draw_object([rect, trajectory_collision_object])
plt.show()
